This notebook is to play around with the VARIMA model.  The model is documented here: <br>
https://unit8co.github.io/darts/ <br>
https://unit8co.github.io/darts/generated_api/darts.models.forecasting.varima.html#darts.models.forecasting.varima.VARIMA

In [17]:
# Imports
import pandas as pd
from darts.models import VARIMA
from darts import TimeSeries
from darts.metrics import mape
from darts.metrics import mae
import optuna

In [7]:
# Load weather data
df = pd.read_csv("weather_interpolation.csv", parse_dates=["date"])
df['date'] = pd.to_datetime(df['date'])
print(df.dtypes)

date                        datetime64[ns]
p (mbar)                           float64
T (degC)                           float64
Tpot (K)                           float64
Tdew (degC)                        float64
rh (%)                             float64
VPmax (mbar)                       float64
VPact (mbar)                       float64
VPdef (mbar)                       float64
sh (g/kg)                          float64
H2OC (mmol/mol)                    float64
rho (g/m**3)                       float64
wv (m/s)                           float64
max. wv (m/s)                      float64
wd (deg)                           float64
rain (mm)                          float64
raining (s)                          int64
SWDR (W/mï¿½)                      float64
PAR (ï¿½mol/mï¿½/s)                float64
max. PAR (ï¿½mol/mï¿½/s)           float64
Tlog (degC)                        float64
OT                                 float64
dtype: object


In [9]:
# Create time series in darts
df = df[['date','T (degC)','VPmax (mbar)','Tdew (degC)']]
series_mv = TimeSeries.from_dataframe(df, time_col='date', freq='10min')

In [34]:
# Determine training and testing sets
# n_train = 10 * 24 * 6  # 10 days / 1440 data points
# n_test = 1 * 24 * 6  # 1 day / 144 data points
n_train = 24 * 6 # 1 day / 144 data points
n_test = 1 * 6 # 1 hour / 6 data points
train, test = series_mv[-(n_train + n_test):-n_test], series_mv[-n_test:]
print(f"Train length: {len(train)}")
print(f"Test length: {len(test)}")
print(f"Train end: {train.end_time()}")
print(f"Test start: {test.start_time()}")

Train length: 144
Test length: 6
Train end: 2020-12-31 23:00:00
Test start: 2020-12-31 23:10:00


In [29]:
# Look at error for basic VARIMA
model = VARIMA()
model.fit(train)
forecast = model.predict(len(test))
print("MAPE:", mape(test, forecast)) # Apparently MAPE doesn't work because of negative numbers

KeyboardInterrupt: 

In [35]:
# Compute error
model = VARIMA()
model.fit(train)
forecast = model.predict(len(test))
test_T_degC = test['T (degC)']
forecast_T_degC = forecast['T (degC)']
print("MAE:", mae(test_T_degC, forecast_T_degC))

MAE: 0.2136173170075326


That ran, and it took about 4 minutes.  Now I'll try optuna to see if I can optimize the parameters.

In [36]:
# Define optuna objectivefunction
def objective(trial):

    # Define the hyperparameter search space for VARIMA parameters
    p = trial.suggest_int('p', 0, 12)  # AR order
    d = trial.suggest_int('d', 0, 1)  # Differencing order
    q = trial.suggest_int('q', 0, 6)  # MA order
    
    # Suggest a trend value from the list of possible trends
    trend = trial.suggest_categorical('trend', ['n', 'c', 't', 'ct'])

    # Print the parameters for this trial
    print(f"Trial {trial.number}: Using p={p}, d={d}, q={q}, trend={trend}")

    # Create and train the VARIMA model with the chosen trend
    model = VARIMA(p=p, d=d, q=q, trend=trend)
    # model.fit(train)
    model.fit(train)

    # Generate predictions on the validation/test set
    forecast = model.predict(len(test))

    # Compute the MAE for the forecast of 'T (degC)'
    error = mae(test_T_degC, forecast['T (degC)'])
    
    return error

In [37]:
# Run optuna
study = optuna.create_study(direction="minimize")  # We want to minimize MAE
study.optimize(objective, n_trials=60) # 60 trials, the basic trial only took 6 seconds

# Print the best hyperparameters 
print(f"Best hyperparameters: {study.best_params}")
print(f"Best MAE: {study.best_value}")

[I 2025-04-05 18:10:01,310] A new study created in memory with name: no-name-ccecfdd0-f383-44fd-943a-0a1e808cd945
c:\Users\aabro\OneDrive\Desktop\SMU Program\Capstone\Python\.venv\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


Trial 0: Using p=7, d=1, q=1, trend=c


[I 2025-04-05 18:11:51,120] Trial 0 finished with value: 0.21730172961089925 and parameters: {'p': 7, 'd': 1, 'q': 1, 'trend': 'c'}. Best is trial 0 with value: 0.21730172961089925.
c:\Users\aabro\OneDrive\Desktop\SMU Program\Capstone\Python\.venv\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


Trial 1: Using p=6, d=1, q=1, trend=ct


c:\Users\aabro\OneDrive\Desktop\SMU Program\Capstone\Python\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\aabro\OneDrive\Desktop\SMU Program\Capstone\Python\.venv\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
[I 2025-04-05 18:12:42,497] Trial 1 finished with value: 0.23599211181522414 and parameters: {'p': 6, 'd': 1, 'q': 1, 'trend': 'ct'}. Best is trial 0 with value: 0.21730172961089925.
c:\Users\aabro\OneDrive\Desktop\SMU Program\Capstone\Python\.venv\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification i

...

Trial 35: Using p=11, d=1, q=6, trend=c


c:\Users\aabro\OneDrive\Desktop\SMU Program\Capstone\Python\.venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
[I 2025-04-05 20:53:07,299] Trial 35 finished with value: 0.07733304630845454 and parameters: {'p': 11, 'd': 1, 'q': 6, 'trend': 'c'}. Best is trial 34 with value: 0.07733304630845454.
c:\Users\aabro\OneDrive\Desktop\SMU Program\Capstone\Python\.venv\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'


Trial 36: Using p=10, d=1, q=6, trend=ct


[W 2025-04-05 20:53:25,568] Trial 36 failed with parameters: {'p': 10, 'd': 1, 'q': 6, 'trend': 'ct'} because of the following error: LinAlgError('Schur decomposition solver error.').
Traceback (most recent call last):
  File "c:\Users\aabro\OneDrive\Desktop\SMU Program\Capstone\Python\.venv\Lib\site-packages\optuna\study\_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\aabro\AppData\Local\Temp\ipykernel_29612\2830198485.py", line 18, in objective
    model.fit(train)
  File "c:\Users\aabro\OneDrive\Desktop\SMU Program\Capstone\Python\.venv\Lib\site-packages\darts\models\forecasting\varima.py", line 130, in fit
    super().fit(series, future_covariates)
  File "c:\Users\aabro\OneDrive\Desktop\SMU Program\Capstone\Python\.venv\Lib\site-packages\darts\models\forecasting\forecasting_model.py", line 3211, in fit
    return self._fit(series, future_covariates=future_covariates)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^

LinAlgError: Schur decomposition solver error.